In [2]:
from SOM import SOM
import sys

sys.path.append("../../")
from utils import load_csv, dataloader, load_save_models
import torch
from torchvision.transforms import v2
import preprocessing.tiff_handling as tiff_handling
import plotly.express as px
from stats import evaluate_perf_utils
import numpy as np

## load in the data

In [3]:
input_dim = 100
map_dim = 80

In [4]:
model = SOM.load_model(SOM(input_dim=input_dim,map_dim=map_dim),'./hpc_SOM_model.pkl')
model.som

Model loaded from ./hpc_SOM_model.pkl.


## load in data

In [5]:
set = load_csv.load_pandas()
train, val,test = load_csv.split_data(set)

print(f'outlier test validation: {any(val["classification"]!=1)}')

/home/bendm/machine_learning/project/majority-vote-outlier-detection/models/SOM/../../utils/load_csv.py:20: DtypeWarning: Columns (91,94,209,213) have mixed types. Specify dtype option on import or set low_memory=False.
  frame = pd.read_csv(data_path, delimiter=";").iloc[:, -3:]


outlier test validation: False


In [6]:
transform = v2.Compose(
    [
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Resize((60, 80)),
        # v2.Lambda(
        #     lambda x: (x.view(-1) - torch.min(x)) / (torch.max(x) - torch.min(x))
        # ),
    ]
)

val_data = np.array(
    [
        tiff_handling.flatten_image(transform(tiff_handling.handle_tiff(el)))
        for el in val["file_names"].tolist()
    ]
)

test["image"] = test["file_names"].apply(lambda x: tiff_handling.flatten_image(transform(tiff_handling.handle_tiff(x))))

In [6]:
# optimize percentile of error threshold
import optuna

truth = [1 if value == 1 else 0 for value in test["classification"]]


def objective(trial):
    perc = trial.suggest_int("percentile", 80,100,step=5)
    model.set_threshold(val_data,perc)
    output = model.predict(test)

    perf_metrics = evaluate_perf_utils.evaluate_performance(truth, output["prediction"])
    return perf_metrics["f1"]


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)

[I 2024-12-09 18:14:26,813] A new study created in memory with name: no-name-d39f22d5-4e78-4248-a86b-c065f0398a6d
[W 2024-12-09 18:15:04,479] Trial 0 failed with parameters: {'percentile': 85} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/bendm/machine_learning/project/majority-vote-outlier-detection/.venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_7019/2331949917.py", line 9, in objective
    model.set_threshold(val_data,perc)
  File "/home/bendm/machine_learning/project/majority-vote-outlier-detection/models/SOM/SOM.py", line 91, in set_threshold
    errors = self.compute_bmu_distance(data)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/bendm/machine_learning/project/majority-vote-outlier-detection/models/SOM/SOM.py", line 73, in compute_bmu_distance
    data[i] - self.som.get_weights()[sel

KeyboardInterrupt: 

In [ ]:
print(f"best percentile: {study.best_params['percentile']}")
print(f"best f1: {study.best_value}")

best percentile: 90
best f1: 0.8363522798251093


In [7]:
model.set_threshold(val_data,percentile=90)

Threshold set at 2.5560 (Percentile: 90)


np.float64(2.5559949231468084)

In [8]:
output = model.predict(test)
output

,classification,file_names,label,image,prediction,error
51,1,/home/bendm/machine_learning/project/majority-...,Singlet,"[tensor(-0.0008), tensor(-0.0019), tensor(0.00...",1,1.844178
6622,1,/home/bendm/machine_learning/project/majority-...,Singlet,"[tensor(0.0059), tensor(-0.0023), tensor(0.002...",1,1.609285
9400,1,/home/bendm/machine_learning/project/majority-...,Singlet,"[tensor(0.0020), tensor(0.0019), tensor(0.0030...",1,2.238647
3022,1,/home/bendm/machine_learning/project/majority-...,Singlet,"[tensor(0.0044), tensor(0.0028), tensor(0.0015...",1,1.588024
1224,1,/home/bendm/machine_learning/project/majority-...,Singlet,"[tensor(0.0042), tensor(0.0017), tensor(0.0014...",1,1.636468
...,...,...,...,...,...,...
153,-1,/home/bendm/machine_learning/project/majority-...,Singlet,"[tensor(-0.0026), tensor(0.0014), tensor(0.012...",0,3.536598
9812,2,/home/bendm/machine_learning/project/majority-...,Doublet,"[tensor(-0.0019), tensor(0.0034), tensor(0.005...",0,3.141779
7403,2,/home/bendm/machine_learning/project/majority-...,Doublet,"[tensor(-0.0089), tensor(0.0029), tensor(0.008...",0,3.411157
7724,-1,/home/bendm/machine_learning/project/majority-...,Singlet,"[tensor(0.0140), tensor(-0.0042), tensor(0.006...",0,3.023635


In [12]:
truth = [0 if value == 1 else 1 for value in output['classification']]
predictions = output['prediction']
predictions = [0 if value == 1 else 1 for value in predictions]

perf_metrics = evaluate_perf_utils.evaluate_performance(truth, predictions)
tn = perf_metrics['tn']
tp = perf_metrics['tp']
fn = perf_metrics['fn']
fp = perf_metrics['fp']
precision = perf_metrics['precision']
recall = perf_metrics['recall']
f1 = perf_metrics['f1']

perf_metrics

{'tn': np.int64(1339),
 'fp': np.int64(118),
 'fn': np.int64(406),
 'tp': np.int64(1051),
 'precision': np.float64(0.8990590248075278),
 'recall': np.float64(0.7213452299245025),
 'f1': np.float64(0.8004569687738005)}

In [13]:
accuracy = (tp + tn) / (tp + tn + fp + fn)
print(f'accuracy: {accuracy}')

accuracy: 0.820178448867536


In [23]:
predictions_partial_scans = output[output["classification"] > 1]["prediction"].tolist()
predictions_partial_scans = [0 if val == 1 else 1 for val in predictions_partial_scans]

partial_truths = np.ones(len(predictions_partial_scans))

parital_perf_metrics = evaluate_perf_utils.evaluate_performance(
    partial_truths, predictions_partial_scans
)
parital_perf_metrics

# print out percentages
print(parital_perf_metrics)
print(f"tp: {parital_perf_metrics['tp']/len(partial_truths)}")
print(f"fn: {parital_perf_metrics['fn']/len(partial_truths)}")

{'tn': np.int64(0), 'fp': np.int64(0), 'fn': np.int64(44), 'tp': np.int64(388), 'precision': np.float64(1.0), 'recall': np.float64(0.8981481481481481), 'f1': np.float64(0.9463414634146341)}
tp: 0.8981481481481481
fn: 0.10185185185185185


In [24]:
predictions_partial_scans = output[output["classification"] < 1]["prediction"].tolist()
predictions_partial_scans = [0 if val == 1 else 1 for val in predictions_partial_scans]

partial_truths = np.ones(len(predictions_partial_scans))

parital_perf_metrics = evaluate_perf_utils.evaluate_performance(
    partial_truths, predictions_partial_scans
)
parital_perf_metrics

# print out percentages
print(parital_perf_metrics)
print(f"tp: {parital_perf_metrics['tp']/len(partial_truths)}")
print(f"fn: {parital_perf_metrics['fn']/len(partial_truths)}")

{'tn': np.int64(0), 'fp': np.int64(0), 'fn': np.int64(362), 'tp': np.int64(663), 'precision': np.float64(1.0), 'recall': np.float64(0.646829268292683), 'f1': np.float64(0.7855450236966824)}
tp: 0.646829268292683
fn: 0.35317073170731705


In [15]:
# Confusion matrix
import plotly.graph_objects as go
import numpy as np

cm = np.array([[tn, fp], [fn, tp]])

total = cm.sum()
cm_percentage = cm / total * 100

# Create a heatmap for the confusion matrix with percentages
fig = go.Figure(
    data=go.Heatmap(
        z=cm_percentage,
        x=["Pred: 0", "Pred: 1"],
        y=["True: 0", "True: 1"],
        colorscale="Peach",  # A more pleasant color palette
        colorbar=dict(title="Percentage", tickvals=[0, 100]),
        showscale=True,
    )
)

# Add titles, labels, and precision, recall, F1 score as annotation
fig.update_layout(
    title="Confusion Matrix (Percentage)",
    xaxis_title="Predicted Labels",
    yaxis_title="True Labels",
    width=500,  # Set the width of the plot
    height=400,  # Set the height of the plot
)

# Add text annotations to show percentage values in each cell
for i in range(cm_percentage.shape[0]):
    for j in range(cm_percentage.shape[1]):
        fig.add_annotation(
            x=j,
            y=i,
            text=f"{cm_percentage[i, j]:.2f}%",
            showarrow=False,
            font=dict(size=14, color="black"),
            align="center",
        )

# Show the plot
fig.show()